# Double Texting

Seamless handling of [double texting](https://langchain-ai.github.io/langgraph/concepts/double_texting/) is important for handling real-world usage scenarios, especially in chat applications.

Users can send multiple messages in a row before the prior run(s) complete, and we want to ensure that we handle this gracefully.

## Reject

A simple approach is to [reject](https://langchain-ai.github.io/langgraph/cloud/how-tos/reject_concurrent/) any new runs until the current run completes.

In [ ]:
%%capture --no-stderr
%pip install -U langgraph_sdk

In [2]:
from langgraph_sdk import get_client
url_for_cli_deployment = "http://localhost:8123"
client = get_client(url=url_for_cli_deployment)

In [3]:
import httpx
from langchain_core.messages import HumanMessage

# Create a thread
thread = await client.threads.create()

# Create to dos
user_input_1 = "Add a ToDo to follow-up with DI Repairs."
user_input_2 = "Add a ToDo to mount dresser to the wall."
config = {"configurable": {"user_id": "Test"}}
graph_name = "task_maistro" 

run = await client.runs.create(
    thread["thread_id"],
    graph_name,
    input={"messages": [HumanMessage(content=user_input_1)]}, 
    config=config,
)
try:
    await client.runs.create(
        thread["thread_id"],
        graph_name,
        input={"messages": [HumanMessage(content=user_input_2)]}, 
        config=config,
        multitask_strategy="reject",
    )
except httpx.HTTPStatusError as e:
    print("Failed to start concurrent run", e)

Failed to start concurrent run Client error '409 Conflict' for url 'http://localhost:8123/threads/b8ab8b4c-36c9-406c-8b80-7d88742537ad/runs'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/409


In [4]:
from langchain_core.messages import convert_to_messages

# Wait until the original run completes
await client.runs.join(thread["thread_id"], run["run_id"])

# Get the state of the thread
state = await client.threads.get_state(thread["thread_id"])
for m in convert_to_messages(state["values"]["messages"]):
    m.pretty_print()

================================ Human Message =================================

Add a ToDo to follow-up with DI Repairs.
================================== Ai Message ==================================
Tool Calls:
  UpdateMemory (call_S37jRn4vnGI9C4p2KaAy4PXc)
 Call ID: call_S37jRn4vnGI9C4p2KaAy4PXc
  Args:
    update_type: todo
================================= Tool Message =================================

New ToDo created:
Content: {'task': 'Follow-up with DI Repairs', 'time_to_complete': 10, 'solutions': ['Call DI Repairs customer service', 'Email DI Repairs support', 'Check DI Repairs website for updates'], 'status': 'not started'}
================================== Ai Message ==================================

I've added "Follow-up with DI Repairs" to your ToDo list. If there's anything else you need, feel free to let me know!


## Enqueue

We can use [enqueue](https://langchain-ai.github.io/langgraph/cloud/how-tos/enqueue_concurrent/https://langchain-ai.github.io/langgraph/cloud/how-tos/enqueue_concurrent/) any new runs until the current run completes.

In [5]:
# Create a new thread
thread = await client.threads.create()

# Create new ToDos
user_input_1 = "Send Erik his t-shirt gift this weekend."
user_input_2 = "Get cash and pay nanny for 2 weeks. Do this by Friday."
config = {"configurable": {"user_id": "Test"}}
graph_name = "task_maistro" 

first_run = await client.runs.create(
    thread["thread_id"],
    graph_name,
    input={"messages": [HumanMessage(content=user_input_1)]}, 
    config=config,
)

second_run = await client.runs.create(
    thread["thread_id"],
    graph_name,
    input={"messages": [HumanMessage(content=user_input_2)]}, 
    config=config,
    multitask_strategy="enqueue",
)

# Wait until the second run completes
await client.runs.join(thread["thread_id"], second_run["run_id"])

# Get the state of the thread
state = await client.threads.get_state(thread["thread_id"])
for m in convert_to_messages(state["values"]["messages"]):
    m.pretty_print()

================================ Human Message =================================

Send Erik his t-shirt gift this weekend.
================================== Ai Message ==================================
Tool Calls:
  UpdateMemory (call_SX7ewyI5vhTq1CIjQhFKFrch)
 Call ID: call_SX7ewyI5vhTq1CIjQhFKFrch
  Args:
    update_type: todo
================================= Tool Message =================================

New ToDo created:
Content: {'task': 'Send Erik his t-shirt gift', 'time_to_complete': 30, 'deadline': '2024-11-17T23:59:59', 'solutions': ['Package the t-shirt', "Print Erik's address", 'Visit the post office']}

Document 16b8b5c8-503a-470c-8b0a-3f23987a5d32 updated:
Plan: Update the status of the task 'Follow-up with DI Repairs' to 'archived' as it is no longer relevant.
Added content: archived
================================== Ai Message ==================================

I've updated your ToDo list to send Erik his t-shirt gift this weekend. Let me know if there's anything 

## Interrupt

We can use [interrupt](https://langchain-ai.github.io/langgraph/cloud/how-tos/interrupt_concurrent/) to interrupt the current run, but save all the work that has been done so far up to that point.


In [6]:
# Create a new thread
thread = await client.threads.create()

# Create new ToDos
user_input_1 = "Order turkey for Thanksgiving by Friday."
user_input_2 = "Never mind, Thanksgiving is the 28th! Order Ham for Thanksgiving by next Friday."
config = {"configurable": {"user_id": "Test"}}
graph_name = "task_maistro" 

interrupted_run = await client.runs.create(
    thread["thread_id"],
    graph_name,
    input={"messages": [HumanMessage(content=user_input_1)]}, 
    config=config,
)

second_run = await client.runs.create(
    thread["thread_id"],
    graph_name,
    input={"messages": [HumanMessage(content=user_input_2)]}, 
    config=config,
    multitask_strategy="interrupt",
)

# Wait until the second run completes
await client.runs.join(thread["thread_id"], second_run["run_id"])

# Get the state of the thread
state = await client.threads.get_state(thread["thread_id"])
for m in convert_to_messages(state["values"]["messages"]):
    m.pretty_print()

================================ Human Message =================================

Never mind, Thanksgiving is the 28th! Order Ham for Thanksgiving by next Friday.
================================== Ai Message ==================================
Tool Calls:
  UpdateMemory (call_xaLVOmrQ2mdXofMkegg8i85u)
 Call ID: call_xaLVOmrQ2mdXofMkegg8i85u
  Args:
    update_type: todo
================================= Tool Message =================================

Document 375d9596-edf8-4de2-985b-bacdc623d6ef updated:
Plan: Update the task to reflect the correct Thanksgiving date and change the task to ordering ham.
Added content: Order Ham for Thanksgiving

New ToDo created:
Content: {'task': 'Call parents back about Thanksgiving plans', 'time_to_complete': 15, 'deadline': None, 'solutions': ['Check calendar for availability', 'Discuss travel arrangements', 'Confirm dinner plans'], 'status': 'not started'}
================================== Ai Message ==================================

I've update

In [7]:
# Confirm that the first run was interrupted
print((await client.runs.get(thread["thread_id"], interrupted_run["run_id"]))["status"])

interrupted


## Rollback

We can use [rollback](https://langchain-ai.github.io/langgraph/cloud/how-tos/rollback_concurrent/) to interrupt the prior run of the graph, delete it, and start a new run with the double-texted input.


In [8]:
# Create a new thread
thread = await client.threads.create()

# Create new ToDos
user_input_1 = "Add a ToDo to call to make appointment at Yoga."
user_input_2 = "Actually, add a ToDo to drop by Yoga in person on Sunday."
config = {"configurable": {"user_id": "Test"}}
graph_name = "task_maistro" 

rolled_back_run = await client.runs.create(
    thread["thread_id"],
    graph_name,
    input={"messages": [HumanMessage(content=user_input_1)]}, 
    config=config,
)

second_run = await client.runs.create(
    thread["thread_id"],
    graph_name,
    input={"messages": [HumanMessage(content=user_input_2)]}, 
    config=config,
    multitask_strategy="rollback",
)

# Wait until the second run completes
await client.runs.join(thread["thread_id"], second_run["run_id"])

# Get the state of the thread
state = await client.threads.get_state(thread["thread_id"])
for m in convert_to_messages(state["values"]["messages"]):
    m.pretty_print()

================================ Human Message =================================

Actually, add a ToDo to drop by Yoga in person on Sunday.
================================== Ai Message ==================================
Tool Calls:
  UpdateMemory (call_5PvCoerSxMzWbioh3jS1US5f)
 Call ID: call_5PvCoerSxMzWbioh3jS1US5f
  Args:
    update_type: todo
================================= Tool Message =================================

New ToDo created:
Content: {'task': 'Drop by Yoga in person', 'time_to_complete': 60, 'deadline': '2024-11-19T23:59:59', 'solutions': ['Check class schedule', 'Prepare yoga outfit', 'Drive to the yoga studio'], 'status': 'not started'}
================================== Ai Message ==================================

I've added "Drop by Yoga in person on Sunday" to your ToDo list. If there's anything else you need, just let me know!


In [9]:
# Confirm that the original run was deleted
try:
    await client.runs.get(thread["thread_id"], rolled_back_run["run_id"])
except httpx.HTTPStatusError as _:
    print("Original run was correctly deleted")

In [11]:
state = await client.runs.get(thread["thread_id"], rolled_back_run["run_id"])
state

{'run_id': '1efa2ca0-e0f3-68a4-a300-73bf756f03c4',
 'thread_id': '2803045f-4266-433b-8c41-f6bcd882cc46',
 'assistant_id': 'ea4ebafa-a81d-5063-a5fa-67c755d98a21',
 'created_at': '2024-11-14T20:50:17.474815+00:00',
 'updated_at': '2024-11-14T20:50:17.474815+00:00',
 'metadata': {},
 'status': 'interrupted',
 'kwargs': {'input': {'messages': [{'id': None,
     'name': None,
     'type': 'human',
     'content': 'Add a ToDo to call to make appointment at Yoga.',
     'example': False,
     'additional_kwargs': {},
     'response_metadata': {}}]},
  'config': {'metadata': {'created_by': 'system'},
   'configurable': {'run_id': '1efa2ca0-e0f3-68a4-a300-73bf756f03c4',
    'user_id': 'Test',
    'graph_id': 'task_maistro',
    'thread_id': '2803045f-4266-433b-8c41-f6bcd882cc46',
    'assistant_id': 'ea4ebafa-a81d-5063-a5fa-67c755d98a21'}},
  'webhook': None,
  'subgraphs': False,
  'temporary': False,
  'stream_mode': ['values'],
  'feedback_keys': None,
  'interrupt_after': None,
  'interrupt